In [ ]:
!pip install langchain-nomic

In [2]:
%pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph tavily-python nomic langchain-text-splitters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pillow-10.3.0-cp39-cp39-macosx_10_10_x86_64.whl.metadata (9.2 kB)
  Using cached aiohttp-3.9.5-cp39-cp39-macosx_10_9_x86_64.whl.metadata (7.5 kB)
  Using cached build-1.2.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached chroma_hnswlib-0.7.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata (252 bytes)
  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached posthog-3.5.0-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached mmh3-4.1.0-cp39-cp39-macosx_10_9_x86_64.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.7 MB/s eta 0:0

In [ ]:
!pip install langchain_huggingface

In [1]:
import os

os.environ["LANGCHAIN_TRACKING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ''

In [2]:
# LLM

local_llm = "llama3"

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_nomic.embeddings import NomicEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://kubernetes.io/ko/docs/concepts/overview/",
    "https://kubernetes.io/ko/docs/tutorials/kubernetes-basics/create-cluster/cluster-intro/",
    "https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4",
    "https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1%20%EC%B1%84%EB%84%90"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
print(len(docs_list))

USER_AGENT environment variable not set, consider setting it to identify your requests.


4


In [3]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250,
    chunk_overlap=0
)

doc_splits = text_splitter.split_documents(docs_list)

In [18]:
# from langchain_huggingface import HuggingFaceEmbeddings

# vectorstore = Chroma.from_documents(
#     documents = doc_splits,
#     collection_name="rag_chroma",
#     embedding = HuggingFaceEmbeddings(
#         model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
#         model_kwargs = {'device':'cpu'}),
# )
# retriever = vectorstore.as_retriever()

/Users/kakao/study/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/kakao/study/lib/python3.9/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/kakao/study/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads 

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = ""

vectorstore = Chroma.from_documents(
    documents = doc_splits,
    collection_name="rag_chroma",
    embedding = OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [160]:
for i in retriever.invoke("네이버는?"):
    print(i)

page_content='되었는가?피드백이 페이지가 도움이 되었나요?네' metadata={'description': '쿠버네티스는 컨테이너화된 워크로드와 서비스를 관리하기 위한 이식할 수 있고, 확장 가능한 오픈소스 플랫폼으로, 선언적 구성과 자동화를 모두 지원한다. 쿠버네티스는 크고 빠르게 성장하는 생태계를 가지고 있다. 쿠버네티스 서비스, 지원 그리고 도구들은 광범위하게 제공된다.\n', 'language': 'ko', 'source': 'https://kubernetes.io/ko/docs/concepts/overview/', 'title': '쿠버네티스란 무엇인가? | Kubernetes'}
page_content='다음 지식인은 사라졌다.[30] 네이버는 약 6조원.[31] 말이 3등이지 점유율이 10%도 되지 않는다.[32] 예를 들면 다음뮤직의 방금그곡 기능이 카카오뮤직으로 이전된 것.[33] 인스턴트 기반의 사진 메신저로, 스냅챗과 유사한' metadata={'language': 'No language found.', 'source': 'https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4', 'title': '카카오 - 나무위키'}
page_content='개념을 최초 도입한 웹 사이트로서, 검색을 비롯해 이메일과 웹서핑 등 인터넷 사용의 장을 본격적으로 마련하고 이끌어내왔다는 점에서 포털계의 대부격이라 할 수 있다. 현재 네이버에 밀리고 있기는 하지만, 한때' metadata={'language': 'No language found.', 'source': 'https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4', 'title': '카카오 - 나무위키'}
page_content='정식으로 대기업으로 분류되었다. IT기업으로서는 국내 최초다.네이버대비 사업 영역 확장이 두드러지다보니 문어발식이라는 비판도 나왔다. 가령 네이버는 쇼핑몰 플랫폼에 도전하다가 쇼핑몰 업계의 항의로 인해'

In [19]:
### Retrieval Grader

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain import hub
import bs4

llm = ChatOpenAI(model="gpt-4o")
prompt = PromptTemplate(
    template="""You are a grader assessing relevance 
    of a retrieved document to a user question. If the document contains keywords related to the user question, 
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n assistant
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()

In [21]:
def run_retrieval_grader(question):
    docs = retriever.invoke(question)
    doc_txt = ""
    for doc in docs:
        doc_txt += doc.page_content
    return retrieval_grader.invoke({"question":question,"document":doc_txt})

In [22]:
question1 = "카카오톡 채널이 뭐야?"
question2 = "네이버 쇼핑이 뭐야?"

print(run_retrieval_grader(question1))
print(run_retrieval_grader(question2))

{'score': 'yes'}
{'score': 'no'}


In [55]:
### Generate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise. 
    and Please contain URL and title at last\
    nQuestion: {question} \nContext: {context} \nAnswer:
    """)
print(prompt)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = prompt | llm |StrOutputParser()

question = "카카오톡 채널이 뭐야?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": format_docs(docs), "question": question})
print(generation)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks. \n    Use the following pieces of retrieved context to answer the question. \n    If you don't know the answer, just say that you don't know. \n    Use three sentences maximum and keep the answer concise. \n    and Please contain URL and title at last    nQuestion: {question} \nContext: {context} \nAnswer:\n    "
카카오톡 채널은 기업들이 홍보와 마케팅을 위해 사용하는 서비스입니다. 여러 기업들이 참가하여 할인 혜택이나 특가 프로모션 등을 광고하며, 아이돌 그룹들도 홍보 목적으로 활용합니다. 

[카카오톡 채널 - 나무위키](https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1%20%EC%B1%84%EB%84%90)


In [263]:
### Hallucination Grader
prompt = PromptTemplate(
    template="""You are a grader assessing whether an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate 
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a 
    single key 'score' and no preamble or explanation. Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    Here is the answer: {generation}
    assistant
    """,
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [259]:
hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": "네이버 채널은 네이버가 운영하는 채널이야"})

'카카오톡 채널의 기능과 활용: 기업 홍보부터 아이돌 그룹까지'

In [35]:
### Answer Grader
prompt = PromptTemplate(
    template="""You are a grader assessing whether an 
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is 
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
    Here is the answer:
    \n ------- \n
    {generation} 
    \n ------- \n
    Here is the question: {question}""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": generation})

{'score': 'yes'}

In [36]:
answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question, "generation": "네이버 채널은 네이버가 운영하는 채널이야"})

{'score': 'no'}

In [42]:
### Router

prompt = PromptTemplate(
    template="""You are an expert at routing a 
    user question to a vectorstore or web search. Use the vectorstore for questions on kakaotalk. You do not need to be stringent with the keywords 
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search' 
    or 'vectorstore' based on the question. Return the a JSON with a single key 'datasource' and 
    no premable or explanation. Question to route: {question}""",
    input_variables=["question"],
)

question_router = prompt | llm | JsonOutputParser()

In [43]:
question = "카카오톡"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

/Users/kakao/study/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'datasource': 'vectorstore'}


In [45]:
question = "오픈채팅"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

{'datasource': 'vectorstore'}


In [176]:
os.environ['TAVILY_API_KEY']=''

In [177]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(k=3)

In [217]:
results = web_search_tool.invoke({"query": "코카콜라 회사는 언제 설립됐어?"})
print(results)

[{'url': 'https://www.coca-cola.com/kr/ko/about-us/history/coca-cola-history-journey-wiki', 'content': '2.1 1880년대. 1886년. 최초의 코카-콜라 탄생 최초의 콜라는 미국 약사 존 펨버턴 (John Pemberton) 에 의해 탄생했다. 탄생 배경을 이해하기 위해서는 당시 시대적 상황을 이해할 필요가 있다. 미국 남북전쟁이 끝나고 가난과 질병에 고통 받는 사람들, 그리고 전후 재건 ...'}, {'url': 'https://www.coca-cola.com/kr/ko/about-us/history', 'content': '코카-콜라의 역사를 이야기할 때 로버트 우드러프를 빼놓고 이야기할 수 없다. 그는 60년간 회사를 이끌며, 코카-콜라를 글로벌 기업으로 성장시켰다. 코카-콜라 역사 페이지에서는 코카-콜라가 어떻게 탄생하고 성장해왔는지, 그리고 세계적인 음료 브랜드로 ...'}, {'url': 'https://ko.wikipedia.org/wiki/코카콜라', 'content': '코카-콜라 ( 영어: Coca-Cola )는 미국 의 코카콜라 컴퍼니 에서 출시한 콜라 (cola) 브랜드이다. 코카콜라는 세계 최초로 만들어진 콜라 [1] 이며, 전 세계적으로 200여 개국 이상에서 팔리고 있는 가장 인지도 높은 상표이기도 하다. 또한 코카콜라는 미국 과 자본주의 ...'}, {'url': 'https://namu.wiki/w/코카콜라', 'content': '콜라(Cola)는 코카콜라뿐이 아니라 펩시 등 다른 회사도 출시하는 음료수 종류이다. 실제로 코카콜라가 처음 만들어졌을 때는 코카나무 잎을 그대로 삶은 베이스에 콜라나무 열매 추출물 [10] 외 각종 향료를 섞어 만든 것이었기에 코카-콜라 [11]가 된 것이었다. 로망스어군 계열 언어를 사용하는 ...'}, {'url': 'https://www.coca-cola.com/kr/ko/offerings/brand-story/

In [254]:
### creator Title

prompt = PromptTemplate(
    template="""You are an korean expert at creating a title. 
    Create title of docs.
    Here are the facts:
    \n ------- \n
    {documents} 
    \n ------- \n
    """,
    input_variables=["documents"],
)

title_creator = prompt | llm

In [229]:
title_creator.invoke({'documents':
               "세상의 모든 관심, 카카오톡 오픈채팅. 카카오톡에서 늘 하던 대화 바로 옆, 오픈채팅탭에서 다양한 관심사와 만나보세요. 나의 요즘 관심사는 물론, 실시간 트렌드와 이슈에 관하여 다양한 사람들과 자유롭게 이야기 나눠보아요!"})

'카카오톡 오픈채팅: 세상의 모든 관심사를 만나는 공간'

In [198]:
type([1])==list

True

In [267]:
from pprint import pprint
from typing import List

from langchain_core.documents import Document
from typing_extensions import TypedDict
from langgraph.graph import END, StateGraph

### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        web_search: whether the add search
        documents: list of documents
    """

    question: str
    generation: str
    web_search: str
    documents: List[str]

### Nodes
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document":d.page_content}
        )
        grade = score["score"]
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    web_search = "Yes" if len(filtered_docs)==0 else "No"
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]
    docs = web_search_tool.invoke({"query":question})
    if type(docs)==list:
        for doc in docs:
            web_results = Document(page_content=doc["content"], 
                                   metadata={'source':doc["url"], 
                                            'title': title_creator.invoke({'documents':doc["content"]})})
            if documents is not None:
                documents.append(web_results)
            else:
                documents = [web_results]
    else:
        pass
    return {"documents": documents, "question": question}

### Conditional edge
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})
    print(source["datasource"])
    if source["datasource"] == "web_search":
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source["datasource"] == "vectorstore":
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """
    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

### Conditional edge
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        return "useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

# Graph Build

In [268]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)
workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("generate", generate)

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")

workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate"
    }
)

workflow.add_edge("websearch", "grade_documents")

workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
    }
)

# Compile
app = workflow.compile()

In [269]:
# Test
inputs = {"question": "사과는 영어로?"}
for output in app.stream(inputs):
    for key,value in output.items():
        pprint(f"Finished runnning: {key}:")
pprint(value["generation"])

---RETRIEVE---
'Finished runnning: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---GRADE: DOCUMENT NOT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---
'Finished runnning: grade_documents:'
---WEB SEARCH---
'Finished runnning: websearch:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
'Finished runnning: grade_documents:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
'Finished runnning: generate:'
('사과는 영어로 "sorry", "apologize", 또는 "pardon"입니다. 상황에 따라 적절한 표현을 선택할 수 있습니다. '
 "[영어로 '사과' 표현하기: Sorry, Apologize, Pardon의 차이점 "
 '이해하기](https://blog-ko.engram.u

In [270]:
# Test
inputs = {"question": "카카오톡 채널이란?"}
for output in app.stream(inputs):
    for key,value in output.items():
        pprint(f"Finished runnning: {key}:")
pprint(value["generation"])

---RETRIEVE---
'Finished runnning: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
'Finished runnning: grade_documents:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
'Finished runnning: generate:'
('카카오톡 채널은 카카오톡 내에서 브랜드나 개인이 소통할 수 있는 공간을 제공하는 서비스입니다. 이를 통해 사용자는 원하는 채널을 '
 '팔로우하고, 해당 채널에서 제공하는 정보나 콘텐츠를 받아볼 수 있습니다. 채널 개설 시 이름과 검색용 아이디를 설정해야 합니다. '
 '[카카오톡 채널 - '
 '나무위키](https://namu.wiki/w/%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%86%A1%20%EC%B1%84%EB%84%90)')
